In [1]:
import importlib
import implementations
importlib.reload(implementations)
from helpers import *
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load data
x_train, x_test, y_train, train_ids, test_ids = load_csv_data('./data/dataset/dataset', sub_sample = False)

In [30]:
def clean_data(x_train, y_train, lim_nans, lim_corr_features, lim_corr_target):

    # Kept fatures
    kept_features = np.array(range(x_train.shape[1]))

    # Remove columns with more than lim_nans of Nans
    percentages = np.sum(np.isnan(x_train), axis = 0) / x_train.shape[0]
    x_tr = x_train[:, percentages < lim_nans]
    kept_features = kept_features[percentages < lim_nans]

    # Remove datapoints (rows) with one or multiple Nans
    keep_idxs = (np.sum(np.isnan(x_tr), axis = 1)) == 0
    x_tr = x_tr[keep_idxs, :]
    y_tr = y_train[keep_idxs] 

    # Remove features with 0 variance
    var = np.var(x_tr, axis = 0)
    x_tr = x_tr[:, var != 0]
    kept_features = kept_features[var != 0]

    # Standardise data along axis 0
    centered_data = x_tr - np.mean(x_tr, axis = 0)
    x_tr = centered_data / np.std(centered_data, axis = 0)

    # Only keep 1 feature among highly correlated features
    corr_tri = np.triu(np.abs(np.corrcoef(x_tr, rowvar = False)), k = 1) # upper triangular correlation matrix (diagonal zeroed as well)
    max_corr = np.max(corr_tri, axis = 0)
    x_tr = x_tr[:, max_corr < lim_corr_features]
    kept_features = kept_features[max_corr < lim_corr_features]

    # Remove features that have very low correlation with target value
    corr_mat = np.abs(np.corrcoef(np.c_[y_tr, x_tr], rowvar = False))
    x_tr = x_tr[:, corr_mat[0, 1:] > lim_corr_target] # first row of correlation matrix indicates correlation with target value vector y
    kept_features = kept_features[corr_mat[0, 1:] > lim_corr_target]

    # Oversample minority class and undersample majority class
    over_coeff = 4.6
    under_coeff = 0.5

    maj_idx = np.where(y_tr == -1)[0]
    min_idx = np.where(y_tr == 1)[0]

    idx_under = np.random.choice(maj_idx, size = int(maj_idx.shape[0] * under_coeff), replace = False)
    idx_over = np.random.choice(min_idx, size = int(min_idx.shape[0] * over_coeff), replace = True)
    idx_balanced = np.concatenate([idx_under, idx_over])
    idx_shuffled = np.random.permutation(idx_balanced)
    x_tr = x_tr[idx_shuffled]
    y_tr = y_tr[idx_shuffled]

    # Make target variable take values in {0,1} instead of {-1,1}, that is map {-1,1} to {0,1}
    y_tr[y_tr == -1] = 0

    # Add offset term to x_tr
    tx_tr = np.c_[np.ones(x_tr.shape[0]), x_tr]

    return tx_tr, y_tr, kept_features


In [ ]:
# Remove outliers (9 typically represents unknown/missing information => remove datapoints with these kind of numbers)
# Replace nans by median values instead of removing datapoints

In [31]:
# Clean data
tx_tr, y_tr, kept_features = clean_data(x_train, y_train, 0.2, 0.95, 0.05)

In [36]:
# Train model with training data (regularized logistic regression)
w, loss = implementations.reg_logistic_regression(y_tr, tx_tr, lambda_ = 0.05, initial_w = np.ones(tx_tr.shape[1]), max_iters = 510, gamma = 0.5)

In [ ]:
# Modify reg_logistic_regression for it to stop if the error becomes stable before the max_iters

In [37]:
# Prepare test data to test model afterwards

# Keep only the features that were kept for the training data
x_te = x_test[:, kept_features]

# Replace nan values with median value for the corresponding feature
medians = np.nanmedian(x_te, axis = 0)
for column in range(x_te.shape[1]):
    x_te[:, column] = np.nan_to_num(x_te[:, column], nan = medians[column])

# Standardise data along axis 0
centered_data = x_te - np.mean(x_te, axis = 0)
x_te = centered_data / np.std(centered_data, axis = 0)

# Add offset term 
tx_te = np.c_[np.ones(x_te.shape[0]), x_te]

In [38]:
# Apply model to test data
pred = np.where(tx_te.dot(w) > 0, 1, -1)
create_csv_submission(test_ids, pred, 'test2')

In [ ]:
#np.max(x_tr, axis = 0)

In [ ]:
"""max_values = np.tile(np.max(x_tr, axis = 0), (x_tr.shape[0], 1))
#np.sum(x_tr[0,:] == np.max(x_tr, axis = 0)) == 0
np.sum(np.sum(x_tr == max_values, axis = 1) <= 5)"""